In [1]:
%%bash
apt update
python -m pip install --upgrade pip
python -m pip install --quiet pandas av protobuf sentencepiece huggingface_hub
python -m pip install --quiet transformers av

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Reading package lists...
Building dependency tree...
Reading state information...
76 packages can be upgraded. Run 'apt list --upgradable' to see them.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


In [12]:
%%bash
pip install --quiet einops einops_exts

# Download datasets

In [3]:
from huggingface_hub import hf_hub_download
import json
import random

In [4]:
hf_hub_download(
    'jwnt4/qvhighlights-val', repo_type='dataset',
    filename='videos.tar.gz', local_dir='.'
)
hf_hub_download(
    'jwnt4/qvhighlights-val', repo_type='dataset',
    filename='highlight_val_release.jsonl', local_dir='.'
)

videos.tar.gz:   0%|          | 0.00/17.6G [00:00<?, ?B/s]

highlight_val_release.jsonl:   0%|          | 0.00/821k [00:00<?, ?B/s]

'highlight_val_release.jsonl'

In [5]:
%%bash
wget https://raw.githubusercontent.com/jayleicn/moment_detr/refs/heads/main/standalone_eval/eval.py

--2025-04-27 10:49:05--  https://raw.githubusercontent.com/jayleicn/moment_detr/refs/heads/main/standalone_eval/eval.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15107 (15K) [text/plain]
Saving to: ‘eval.py’

     0K .......... ....                                       100% 4.38M=0.003s

2025-04-27 10:49:05 (4.38 MB/s) - ‘eval.py’ saved [15107/15107]



In [6]:
%%bash
mkdir videos/
tar -xzf videos.tar.gz -C videos/

In [7]:
with open('highlight_val_release.jsonl', 'r') as f_in:
    lines = f_in.read().strip().split('\n')
val_jsonl = []
for line in lines:
    val_jsonl.append(line)

val_jsonl2 = random.sample(val_jsonl, 250)
val_jsonl2_str = '\n'.join(val_jsonl2)
with open('highlight_val_small.jsonl', 'w') as f_out:
    f_out.write(val_jsonl2_str)

val_jsonl = []
for line in val_jsonl2:
    val_jsonl.append(json.loads(line))
with open('highlight_val_small.json', 'w') as f_out:
    json.dump(val_jsonl, f_out)

# xgen-mm try best prompt

In [1]:
# import stuff

import pandas as pd
import torch
import torch.nn as nn
import torch.functional as F
import numpy as np
import numpy as np
import torchvision
import torchvision.io
from huggingface_hub import hf_hub_download
import math
import json
import random

In [2]:
# load small set (250 questions instead of 1.5k)

with open('highlight_val_small.json', 'r') as f_in:
    val_jsonl = json.load(f_in)

val_jsonl[0]

{'qid': 2988,
 'query': 'mum is talking with girls and putting shoes on girl.',
 'duration': 150,
 'vid': 'H3fhZxUC5M8_360.0_510.0',
 'relevant_clip_ids': [62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74],
 'saliency_scores': [[2, 4, 3],
  [4, 4, 4],
  [4, 4, 3],
  [3, 4, 3],
  [3, 4, 3],
  [2, 4, 3],
  [2, 4, 3],
  [2, 4, 3],
  [2, 4, 3],
  [2, 4, 4],
  [2, 4, 4],
  [2, 4, 3],
  [2, 4, 3]],
 'relevant_windows': [[124, 150]]}

In [3]:
from transformers import AutoModelForVision2Seq, AutoTokenizer, AutoImageProcessor, LogitsProcessor
import torch

model_name_or_path = "Salesforce/xgen-mm-vid-phi3-mini-r-v1.5-128tokens-8frames"
model = AutoModelForVision2Seq.from_pretrained(model_name_or_path, trust_remote_code=True, torch_dtype=torch.float32)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True, use_fast=False, legacy=False)
image_processor = AutoImageProcessor.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer = model.update_special_tokens(tokenizer)

model = model.to('cuda', dtype=torch.float32)  # even if the model is loaded in float32, some params can be downcasted to bfloat16 which wont work
model.eval()
tokenizer.padding_side = "left"
tokenizer.eos_token = "<|end|>"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
!nvidia-smi

Sun Apr 27 11:49:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.58.02              Driver Version: 555.58.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:01:00.0 Off |                  Off |
|  0%   54C    P0             70W /  450W |   17548MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# check if there are some params that is still in bfloat16 or fp16
for name, p in model.named_parameters():
    if p.dtype != torch.float32:
        print(f"{name:40} --> {p.dtype}")

In [6]:
prompt_template1 = '''You are given a {frame}-frame video. At which frame do you see the activity: "{activity}"? \
Answer only with comma separated numbers denoting the frame numbers. \
Do not provide any other explanations!'''

prompt_template2 = '''You are given a {frame}-frame video. Provide a comma-separated numbers for frames that matches with the activity: "{activity}". \
Do not provide any other explanations.'''

prompt_template3 = '''- Instruction: analyze and find the relevant frames of the given Query.
- Output format: comma-separated of the relevant frame number (e.g. 1,5,6).
- Constraint: Do not provide any other explanations!
- Number of frames: {frame}.
- Query: {activity}.
Question: What frames are relevant with the given Query?'''

prompt_template4 = '''You are given a {frame}-frame video. At which frames you see {activity}? \
Provide only frame numbers separated by comma. Example of output format: "1,2,3,10". Do not provide any other explanations!'''

prompt_templates = [
    prompt_template1, prompt_template2, prompt_template3, prompt_template4
]

In [19]:
def sample_frames(vframes, num_frames, fps):
    frame_indice = np.linspace(int(num_frames/2), len(vframes) - int(num_frames/2), num_frames, dtype=int)
    video = vframes[frame_indice]
    video_list = []
    for i in range(len(video)):
        video_list.append(torchvision.transforms.functional.to_pil_image(video[i]))
    timestamps = (frame_indice / fps).tolist() 
    return video_list, list(map(round, timestamps))

def generate(messages, images):
    # img_bytes_list = [base64.b64decode(image.encode("utf-8")) for image in images]
    # images = [Image.open(BytesIO(img_bytes)) for img_bytes in img_bytes_list]
    image_sizes = [image.size for image in images]
    # Similar operation in model_worker.py
    image_tensor = [image_processor([img])["pixel_values"].to(model.device, dtype=torch.float32) for img in images]

    image_tensor = torch.stack(image_tensor, dim=1)
    image_tensor = image_tensor.squeeze(2)
    inputs = {"pixel_values": image_tensor}

    full_conv = "<|system|>\nA chat between a curious user and an artificial intelligence assistant.\
    The assistant gives helpful, detailed, and polite answers to the user's questions regarding the video.<|end|>\n"
    for msg in messages:
        msg_str = "<|{role}|>\n{content}<|end|>\n".format(
            role=msg["role"], content=msg["content"]
        )
        full_conv += msg_str

    full_conv += "<|assistant|>\n"
    # print(full_conv)
    language_inputs = tokenizer([full_conv], return_tensors="pt")
    for name, value in language_inputs.items():
        language_inputs[name] = value.to(model.device)
    inputs.update(language_inputs)
    # print(inputs)

    with torch.inference_mode():
        generated_text = model.generate(
            **inputs,
            image_size=[image_sizes],
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            temperature=0.05,
            do_sample=False,
            max_new_tokens=1024,
            top_p=None,
            num_beams=1,
        )

    outputs = (
        tokenizer.decode(generated_text[0], skip_special_tokens=True)
        .split("<|end|>")[0]
        .strip()
    )
    return outputs


def predict(qvh, num_frames=20, prompt_template=1):
    video_file = f"videos/{qvh['vid']}.mp4"
    
    vframes, _, info = torchvision.io.read_video(
        filename=video_file, pts_unit="sec", output_format="TCHW"
    )
    fps = float(info['video_fps'])
    total_frames = len(vframes)
    images, timestamps = sample_frames(vframes, num_frames, fps)

    prompt = ""
    prompt = prompt + "<image>\n"
    # prompt = prompt + "What's the main gist of the video ?"
    # prompt = prompt + "Please describe the primary object or subject in the video, capturing their attributes, actions, positions, and movements."
    prompt = prompt + prompt_templates[prompt_template-1].format(frame=num_frames, activity=qvh['query'])
    messages = [{"role": "user", "content": prompt}]
    generated = generate(messages, images)
    return generated, timestamps

## val_jsonl[2]

In [8]:
preds = predict(val_jsonl[2], prompt_template=1)
print(preds[0])
print(val_jsonl[2]['relevant_windows'])

/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.05` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


The activity "A woman in a black dress giving a tour of her hotel room" is seen at frame 19.
[[96, 130]]


In [9]:
preds = predict(val_jsonl[2], prompt_template=2)
print(preds[0])
print(val_jsonl[2]['relevant_windows'])

1, 2, 3, 4, 5, 6, 7, 8, 9, 10
[[96, 130]]


In [10]:
preds = predict(val_jsonl[2], prompt_template=3)
print(preds[0])
print(val_jsonl[2]['relevant_windows'])

The relevant frames are 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, and 20.
[[96, 130]]


In [11]:
preds = predict(val_jsonl[2], prompt_template=4)
print(preds[0])
print(val_jsonl[2]['relevant_windows'])

The woman in a black dress is giving a tour of her hotel room at frames 1, 2, 3, and 10.
[[96, 130]]


## val_jsonl[10]

In [12]:
for i in range(4):
    preds = predict(val_jsonl[10], prompt_template=i+1)
    print(preds[0])
    print(val_jsonl[10]['relevant_windows'])

The activity "Old man speaks next to plaques" is seen at frame 18.
[[112, 128]]
10
[[112, 128]]
The relevant frames with the given Query are 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, and 20.
[[112, 128]]
The frames where Old man speaks next to plaques are 1, 2, 3, and 10.
[[112, 128]]


## prompts from chatgpt

In [13]:
new_prompts = [
'''Example: Video of 5 frames, activity “clap” → 1,3  
You have a {frame}-frame video. Identify all frames with “{activity}”.  
Strictly output only a comma-separated list of frame numbers. No other text.''',
'''Example: 5-frame video, “sit” → 2,5  
Task: find frames in a {frame}-frame video where “{activity}” occurs.  
Output exactly “n,n,…”—no words, quotes or punctuation beyond commas.''',
    
'''Demo: 4 frames, “jump” → 2,4  
Instruction: From a {frame}-frame video, list all frames showing “{activity}”.  
Answer format: comma-separated numbers only. Do NOT add explanations.''',
    
'''Example: 6 frames, “wave” → 1,6  
You are given a video of {frame} frames and the activity "{activity}".  
Strictly return only the frame numbers separated by commas (e.g. 1,5,10).''',
    
'''Example: Video of 5 frames, activity “clap”: → 1,3

You are given a video of {frame} frames.  
Task: identify all frames showing “{activity}”.  
Strictly output exactly a comma-separated list of frame numbers (e.g. 1,5,10).  
Do NOT include any other words, punctuation or explanation.''',
]

prompt_templates = new_prompts

In [14]:
for i in range(5):
    preds = predict(val_jsonl[10], prompt_template=i+1)
    print(preds[0])
    print(val_jsonl[10]['relevant_windows'])

1, 2, 3, 4, 5
[[112, 128]]
The video is about a man who is talking to the camera.
[[112, 128]]
1, 2, 3, 4
[[112, 128]]
1,2,3,4,5,6,7,8,9,10
[[112, 128]]
1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
[[112, 128]]


## label boxes prompt from chatgpt:

In [20]:
prompt_templates = [
'''Example: 20-frame video, “man sitting down” → 00110  
You have a {frame}-frame video.  
Task: output a binary string of length {frame} where each “1” marks a frame with “{activity}” and “0” marks all others.  
Do NOT add spaces, commas, or any extra text.''',

'''Demo: 7 frames, “man sitting down” → 0001001  
Instruction: For a {frame}-frame video, produce exactly {frame} digits (0/1) indicating frames with “{activity}”.  
No separators or explanations—only the digit sequence.''',
    
'''Example: frames=20, activity=“man sitting down”, answer=12222111111111111221  
Question: You’re given {frame} frames video and the activity “{activity}”.  
Output exactly {frame} characters of 1 or 2, with 2 for frames containing the activity.  
No quotes, no text, no line breaks.''',

'''Demo: frames=20, activity=“man sitting down”, answer=12222111111111111221  
Question: You have a video of {frame} frames video. Mark frames showing “{activity}” with “2” and others with “1”.  
Return only the continuous 0/1 string of length {frame}.'''
]

In [21]:
for i in range(4):
    preds = predict(val_jsonl[10], prompt_template=i+1)
    print(preds[0])
    print(val_jsonl[10]['relevant_windows'])

00110
[[112, 128]]
000100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100100

## better prompt version

In [17]:
def predict(qvh, num_frames=20, prompt_template=1):
    video_file = f"videos/{qvh['vid']}.mp4"
    
    vframes, _, info = torchvision.io.read_video(
        filename=video_file, pts_unit="sec", output_format="TCHW"
    )
    fps = float(info['video_fps'])
    total_frames = len(vframes)
    images, timestamps = sample_frames(vframes, num_frames, fps)

    prompt = ""
    prompt = prompt + "<image>\n"
    # prompt = prompt + "What's the main gist of the video ?"
    # prompt = prompt + "Please describe the primary object or subject in the video, capturing their attributes, actions, positions, and movements."
    demo = '1'*(num_frames//2) + '2'*(num_frames - num_frames//2)
    prompt = prompt + prompt_templates[prompt_template-1].format(frame=num_frames, activity=qvh['query'], demo=demo)
    messages = [{"role": "user", "content": prompt}]
    generated = generate(messages, images)
    return generated, timestamps

prompt_templates = [
'''Demo: {frame}-frame video, activity "{activity}" → {demo}
Hey, there’s a {frame}-frame clip and I’m looking for moments of "{activity}". 
Mark each frame: use "2" if the activity shows up, "1" if not.
Send back exactly {frame} digits, no quotes or extra words.''',
'''
Demo: {frame}-frame video, activity "{activity}" → {demo}
Hey, there’s a {frame}-frame clip and I’m looking for moments of "{activity}". 
Mark each frame: use "2" if the activity shows up, "1" if not.
Send back exactly {frame} digits, no quotes or extra words.

Q: Which frames show “{activity}”?
A:"''',
    
'''Me: Here’s a {frame}-frame clip and I want to spot “{activity}”. 
Provide your answers like this: frames → {demo}
You:''',
]

In [18]:
for i in range(3):
    preds = predict(val_jsonl[10], prompt_template=i+1)
    print(preds[0])
    print(val_jsonl[10]['relevant_windows'])

1
[[112, 128]]
The frames that show "Old man speaks next to plaques" are:
[[112, 128]]
The old man speaks next to plaques.
[[112, 128]]


# xgen-mm interleaved

In [3]:
# import stuff

import pandas as pd
import torch
import torch.nn as nn
import torch.functional as F
import numpy as np
import numpy as np
import torchvision
import torchvision.io
from huggingface_hub import hf_hub_download
import math
import json
import random
from transformers import AutoModelForVision2Seq, AutoTokenizer, AutoImageProcessor, StoppingCriteria

# load small set (250 questions instead of 1.5k)

with open('highlight_val_small.json', 'r') as f_in:
    val_jsonl = json.load(f_in)

val_jsonl[0]

{'qid': 2988,
 'query': 'mum is talking with girls and putting shoes on girl.',
 'duration': 150,
 'vid': 'H3fhZxUC5M8_360.0_510.0',
 'relevant_clip_ids': [62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74],
 'saliency_scores': [[2, 4, 3],
  [4, 4, 4],
  [4, 4, 3],
  [3, 4, 3],
  [3, 4, 3],
  [2, 4, 3],
  [2, 4, 3],
  [2, 4, 3],
  [2, 4, 3],
  [2, 4, 4],
  [2, 4, 4],
  [2, 4, 3],
  [2, 4, 3]],
 'relevant_windows': [[124, 150]]}

In [ ]:
model_name_or_path = "Salesforce/xgen-mm-phi3-mini-instruct-interleave-r-v1.5"
model = AutoModelForVision2Seq.from_pretrained(model_name_or_path, trust_remote_code=True, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True, use_fast=False, legacy=False) 
image_processor = AutoImageProcessor.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer = model.update_special_tokens(tokenizer)

In [5]:
model = model.to('cuda')
model.eval()
tokenizer.padding_side = "left"
tokenizer.eos_token = '<|end|>'

In [35]:
import json
import PIL
import textwrap
import IPython.display as display
from IPython.display import Image

def apply_prompt_template(prompt):
    s = (
                '<|system|>\nA chat between a curious user and an artificial intelligence assistant. '
                "Given an activity, the assistant analyzes the images and outputs which image has high probability of containing the activity. \
                Then, the assistant outputs a 15 character binary mask with '1' as low probability and '2' as high probability. \
                Example: 111222211111111<|end|>\n"
                f'<|user|>\n{prompt}<|end|>\n<|assistant|>\n'
            )
    return s 

prompt_templates = [
'''You are given {frame} images that are sampled from a video starting from image 1, image 2, and until image {frame}.
Your task is to identify which frames are likely to show {activity}.
Output exactly {frame} characters of 1 or 2, with 2 for frames containing the activity and 1 not. 
No quotes, no text, no line breaks.
Example: If you think the first and second frame contains the activity, answer 22111''',

'''Example: frames={frame}, activity="man sitting down in Bus", answer={demo}
You are given {frame} images sampled from a video in order from image 1 to image {frame+1}.
Identify the frames where {activity} is happening.
Output exactly {frame} characters using 1 or 2: write 2 if the activity appears, and 1 if it does not.
No quotes, no extra text, no line breaks.''',

'''Example: frames={frame}, activity="man sitting down in Bus", answer={demo}
You are provided with {frame} sequential images taken from a video, numbered from image 1 to image {frame+1}.
Decide which frames show {activity}.
Write a sequence of {frame} characters: 2 for frames showing the activity, and 1 otherwise.
No quotes, no extra words, no line breaks.''',

'''Example: frames={frame}, activity="man sitting down in Bus", answer={demo}
You are shown {frame} frames sampled from a video, starting from image 1 to image {frame+1}.
Your task is to detect {activity} across the frames.
Output exactly {frame} characters, marking 2 if the activity is present, and 1 if not.
No quotes, no explanations, no line breaks.''',
'''
Activity: {activity} Which images have high probability of containing the activity? Output exactly {frame} characters using 1 or 2: write 2 if the activity appears, and 1 if it does not.'''
]

def make_prompt(frame, activity, prompt_idx, timestamps):
    demo = '1'*(frame//2) + '2'*(frame - frame//2)
    prompt = ("<image>"*frame) + '\n' + prompt_templates[prompt_idx-1].format(activity=activity, frame=frame, demo=demo) 
    
    return prompt

In [8]:
def sample_frames(vframes, num_frames, fps):
    frame_indice = np.linspace(int(num_frames/2), len(vframes) - int(num_frames/2), num_frames, dtype=int)
    video = vframes[frame_indice]
    video_list = []
    for i in range(len(video)):
        video_list.append(torchvision.transforms.functional.to_pil_image(video[i]))
    timestamps = (frame_indice / fps).tolist() 
    return video_list, list(map(round, timestamps))

In [19]:
def predict_interleave(qvh, num_frames=15, template_idx=1):
    video_file = f"videos/{qvh['vid']}.mp4"
    vframes, _, info = torchvision.io.read_video(
        filename=video_file, pts_unit="sec", output_format="TCHW"
    )
    fps = float(info['video_fps'])
    total_frames = len(vframes)
    images, timestamps = sample_frames(vframes, num_frames, fps)
    image_list = []
    for index, img in enumerate(images):
        # display.display(img)
        image_list.append(image_processor([img], image_aspect_ratio='anyres')['pixel_values'].cuda())
    image_sizes = [image.size for image in images]
    inputs = {
        "pixel_values": [image_list]
    }
    # prompt = apply_prompt_template("What happens in these images chronologically?" + ("<image>"*num_frames))
    prompt = apply_prompt_template(make_prompt(num_frames, qvh['query'], template_idx, timestamps))
    language_inputs = tokenizer([prompt], return_tensors="pt")
    inputs.update(language_inputs)
    
    for name, value in inputs.items():
        if isinstance(value, torch.Tensor):
            inputs[name] = value.cuda()
    generated_text = model.generate(**inputs, image_size=[image_sizes],
                                    pad_token_id=tokenizer.pad_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    temperature=0.05,
                                    do_sample=False, max_new_tokens=1024, top_p=None, num_beams=1,
                                    )
    prediction = tokenizer.decode(generated_text[0], skip_special_tokens=True).split("<|end|>")[0]
    print("User: ", prompt)
    print("Assistant: ", textwrap.fill(prediction, width=100))

In [15]:
predict_interleave(val_jsonl[0])

User:  <|system|>
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.<|end|>
<|user|>
What happens in these images chronologically?<image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><|end|>
<|assistant|>

Assistant:  a group of children playing in a room, with a woman standing nearby. The children are wearing
colorful clothing, and there are toys and furniture in the room. The woman is interacting with the
children, possibly playing with them or supervising their activities. The children are engaged in
various activities, such as playing with toys, sitting on the floor, and standing. The woman is
wearing a plaid shirt and appears to be in a relaxed and comfortable position. The children are
wearing colorful clothing, and there are toys and furniture in the room. The woman is interacting
with the children, possibly playing with them

In [36]:
predict_interleave(val_jsonl[0], template_idx=5)

User:  <|system|>
A chat between a curious user and an artificial intelligence assistant. Given an activity, the assistant analyzes the images and outputs which image has high probability of containing the activity.                 Then, the assistant outputs a 15 character binary mask with '1' as low probability and '2' as high probability.                 Example: 111222211111111<|end|>
<|user|>
<image><image><image><image><image><image><image><image><image><image><image><image><image><image><image>

Activity: mum is talking with girls and putting shoes on girl. Which images have high probability of containing the activity? Output exactly 15 characters using 1 or 2: write 2 if the activity appears, and 1 if it does not.<|end|>
<|assistant|>

Assistant:  1
